# Some Typical Financial Calculations

# Load Packages

The Roots and Distributions packages have been used in the previous notebooks, but StatsBase not. It contains methods for estimating means, variances, ranks, correlations etc. See https://github.com/JuliaStats/StatsBase.jl. 

In [1]:
using StatsBase, Roots, Distributions

include("printmat.jl")            #function for prettier matrix printing

println4Ps (generic function with 1 method)

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# CAPM and Return Autocorrelation

The CAPM regression is

$R_{it}^{e}  =\alpha_{i}+\beta_{i}R_{mt}^{e}+\varepsilon_{it}$,

where $R_{mt}^{e}$ is the market excess return. Theory says that $\alpha=0$, which is easily tested.

### Load Data

In [3]:
xx  = readdlm("Data/MyData.csv",',',header=true)        #monthly return data
x   = xx[1]
ym  = x[:,1]                 #yearmonth
Rme = x[:,2]                 #market excess return
Rf  = x[:,3]                 #interest rate
R   = x[:,4]                 #return small growth stocks
Re  = R - Rf                 #excess returns
T   = size(Rme,1)

388

### CAPM regression

In [4]:
x    = [ones(T) Rme]             #regressors
y    = copy(Re)                  #to get standard OLS notation
b    = x\y                       #OLS
u    = y - x*b                   #residuals
covb = inv(x'x)*var(u)           #cov(b), see any textbook
stdb = sqrt.(diag(covb))         #std(b)
R2a  = 1 - var(u)/var(y)

println("     coeff      std     t-stat")
printmat([b stdb b./stdb])
printlnPs("R2: ",R2a)
printlnPs("no. of observations: ",T)

     coeff      std     t-stat
    -0.504     0.304    -1.656
     1.341     0.066    20.427

R2:      0.519
no. of observations:        388


## Return Autocorrelations

That is, the correlation of $R_{t}^{e}$ and $R_{t-s}^{e}$. 

(It is easily shown that the t-stat of an autocorrelation is $\sqrt{T}$ times the autocorrelation.)

In [5]:
plags = 1:5
xCorr = autocor(Re,plags)         #using the StatsBase package

println("Autocorrelations of the excess returns in Re")
println("\n     lag   autocorrr    t-stat")
printmat([plags xCorr sqrt(T)*xCorr])

Autocorrelations of the excess returns in Re

     lag   autocorrr    t-stat
     1.000     0.216     4.253
     2.000     0.002     0.046
     3.000    -0.018    -0.359
     4.000    -0.065    -1.289
     5.000    -0.027    -0.536



# Value at Risk

The next cell first constructs an simple estimate of $\sigma_t^2$ as a backward looking moving average (the RiskMetrics approach)

$\sigma_t^2 = \lambda \sigma_{t-1}^2 + (1-\lambda) (R_{t-1} -\mu)^2$,
where $\mu$ is the average return (for all data).

Then, we calculate the 95% VaR by assuming a $N(\mu,\sigma_t^2)$ distribution

$\textrm{VaR}_{t} = - (\mu-1.64\sigma_t)$.

If the model is correct, then the loss should not exceed $\textrm{VaR}_{t}$ in more than 5% of the months.

In [6]:
μ = mean(Rme)

λ = 0.95                         #weight on old volatility
σ² = fill(var(Rme),T)                 #RiskMetrics approach to estimate variance     
for t = 2:T
  σ²[t] = λ*σ²[t-1] + (1-λ)*(Rme[t-1]-μ)^2    
end

VaR95 = -(μ - 1.64*sqrt.(σ²))      #VaR at 95% level

println("let's plot it")

let's plot it


In [7]:
YearFrac = floor.(ym/100) + (mod.(ym,100)-1)/12    #eg 1990.92 for Dec 1990

plot1 = plot(YearFrac,VaR95,color=:blue,legend=false)
plot!(xlims=(1978,2012),ylims=(0,11))
title!("1-month Value at Risk (95%) for US equity market")
ylabel!("%")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 1-month Value at Risk (95%) for US equity market 
 
 
 % 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.6276,140.694 56.9824,143.157 58.3372,144.542 59.692,142.781 61.0468,148.904 62.4016,152.578 63.7564,155.189 65.1112,161.084 66.466,158.508 67.8208,163.787 
 69.1755,145.534 70.5303,144.832 71.8851,150.506 73.2399,148.37 74.5947,153.846 75.9495,105.666 77.3043,109.86 78.6591,111.334 80.0139,116.599 81.3687,114.423 
 82.7235,121.018 84.0783,127.076 85.4331,133.63 86.7879,117.809 88.1427,116.917 89.4975,115.196 90.8522,122.106 92.207,126.656 93.5618,131.048 94.9166,137.516 
 96.2714,141.266 97.6262,146.134 98.981,135.572 100.336,122.872 101.691,124.689 103.045,128.994 104.4,130.367 105.755,132.288 107.11,126.337 108.465,131.073 
 109.819,135.646 111.174,136.214 112.529,138.043 113.884,140.35 115.239,115.423 116.593,122.243 117.948,98.9872 119.303,102.323 120.658,109.554 122.013,114.385 
 123.367,120.445 124.722,125.843 126.077,122.152 127.432,128.889 128.786,133.664 130.141,134.275 131.496,140.407 132.851,146.664 134.206,147.523 135.56,152.772 
 136.915,156.955 138.27,160.551 139.625,157.606 140.98,163.446 142.334,168.693 143.689,159.964 145.044,165.427 146.399,167.071 147.754,142.353 149.108,147.97 
 150.463,153.253 151.818,157.39 153.173,162.827 154.528,151.572 155.882,157.484 157.237,162.705 158.592,167.635 159.947,167.042 161.302,172.539 162.656,177.462 
 164.011,181.864 165.366,177.663 166.721,179.446 168.075,173.473 169.43,175.707 170.785,181.077 172.14,173.342 173.495,172.816 174.849,177.016 176.204,176.854 
 177.559,182.175 178.914,170.044 180.269,165.318 181.623,145.767 182.978,147.375 184.333,153.391 185.688,154.982 187.043,120.077 188.397,122.99 189.752,129.236 
 191.107,133.651 192.462,140.032 193.817,143.108 195.171,145.928 196.526,149.943 197.881,152.944 199.236,22.0812 200.591,18.0442 201.945,20.1874 203.3,26.9078 
 204.655,32.6104 206.01,40.0963 207.365,48.9111 208.719,57.252 210.074,61.9643 211.429,69.4556 212.784,73.8427 214.138,80.335 215.493,88.0589 216.848,93.7129 
 218.203,100.976 219.558,100.663 220.912,105.855 222.267,112.803 223.622,116.409 224.977,121.54 226.332,127.392 227.686,122.58 229.041,129.096 230.396,135.096 
 231.751,136.363 233.106,142.671 234.46,148.773 235.815,134.955 237.17,141.338 238.525,147.18 239.88,148.115 241.234,136.956 242.589,142.514 243.944,147.251 
 245.299,121.852 246.654,116.807 248.008,121.934 249.363,120.686 250.718,126.615 252.073,129.205 253.428,123.952 254.782,129.688 256.137,136.05 257.492,139.807 
 258.847,137.476 260.201,140.048 261.556,145.551 262.911,150.273 264.266,156.119 265.621,154.983 266.975,132.733 268.33,138.876 269.685,145.127 271.04,147.961 
 272.395,153.991 273.749,159.904 275.104,163.059 276.459,165.663 277.814,168.434 279.169,173.952 280.523,179.353 281.878,181.065 283.233,186.022 284.588,191.075 
 285.943,196.038 287.297,199.846 288.652,200.169 290.007,203.137 291.362,207.785 292.717,212.005 294.071,212.476 295.426,216.655 296.781,220.578 298.136,221.833 
 299.491,225.491 300.845,227.223 302.2,226.562 303.555,217.567 304.91,221.886 306.264,226.098 307.619,224.056 308.974,226.084 310.329,225.321 311.684,225.888 
 313.038,229.898 314.393,223.922 315.748,228.061 317.103,231.637 318.458,231.514 319.812,234.218 321.167,237.098 322.522,238.444 323.877,240.175 325.232,239.403 
 326.586,243.161 327.941,243.399 329.296,244.606 330.651,242.898 332.006,246.485 333.36,248.43 334.715,251.754 336.07,255.205 337.425,257.345 338.78,259.111 
 340.134,260.468 341.489,241.648 342.844,242.848 344.199,237.566 345.553,241.321 346.908,230.268 348.263,232.008 349.618,227.435 350.973,230.948 352.327,221.173 
 353.682,220.898 355.037,209.562 356.392,209.248 357.747,196.492 359.101,193.193 360.456,

# Options

## Black-Scholes Option Price

Let $S$ be the the current spot price of an asset and $y$ be the interest rate.

The Black-Scholes formula for a European call option (on that asset) with strike price $K$ and time to expiration $m$ is

$C  =S\Phi\left(  d_{1}\right)  -e^{-ym}K\Phi\left(d_{2}\right)$, where

$d_{1} =\frac{\ln(S/K)+(y+\sigma^{2}/2)m}{\sigma\sqrt{m}} \ \text{ and } \ d_{2}=d_{1}-\sigma\sqrt{m}$ 

and where $\Phi(d)$ denotes the probability of $x\leq d$ when $x$ has an $N(0,1)$ distribution. All variables except the volatility ($\sigma$) are directly observable. 

In [8]:
function Φ(x)
    #Calculates Pr(z<=x) for N(0,1) variable z
    Pr = cdf(Normal(0,1),x)         #Distributions package
  return Pr
end

function OptionBlackSPs(S,K,m,y,σ)
#Calculates Black-Scholes european call option price
  d1 = ( log.(S./K) + (y+1/2*σ.^2)*m ) ./ (σ*sqrt(m))
  d2 = d1 - σ*sqrt(m)
  c  = S.*Φ(d1) - exp(-y*m).*K.*Φ(d2)
  return c
end

OptionBlackSPs (generic function with 1 method)

In [9]:
σ = 0.4
c1 = OptionBlackSPs(10,10,0.5,0.1,σ)
printlnPs("\n","call price according to Black-Scholes: ",c1)

K = linspace(7,13,51)
c = OptionBlackSPs(10,K,0.5,0.1,σ)

println("let's plot it")


call price according to Black-Scholes:      1.358
let's plot it


In [10]:
plot1 = plot(K,c,color=:red,legend=false)
title!("Black-Scholes call option price (for different strike prices)")
xlabel!("strike price")
ylabel!("option price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 7 
 
 
 8 
 
 
 9 
 
 
 10 
 
 
 11 
 
 
 12 
 
 
 13 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 Black-Scholes call option price (for different strike prices) 
 
 
 strike price 
 
 
 option price 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,7.87402 50.4252,20.1488 61.4803,32.2721 72.5354,44.2355 83.5906,56.0307 94.6457,67.6498 105.701,79.0852 116.756,90.3296 127.811,101.376 138.866,112.219 
 149.921,122.852 160.976,133.27 172.031,143.467 183.087,153.441 194.142,163.186 205.197,172.701 216.252,181.982 227.307,191.027 238.362,199.835 249.417,208.405 
 260.472,216.737 271.528,224.83 282.583,232.685 293.638,240.304 304.693,247.686 315.748,254.836 326.803,261.753 337.858,268.442 348.913,274.906 359.969,281.146 
 371.024,287.168 382.079,292.974 393.134,298.569 404.189,303.957 415.244,309.142 426.299,314.13 437.354,318.924 448.409,323.529 459.465,327.951 470.52,332.194 
 481.575,336.263 492.63,340.163 503.685,343.9 514.74,347.478 525.795,350.903 536.85,354.179 547.906,357.312 558.961,360.306 570.016,363.166 581.071,365.897 
 592.126,368.504 
 "/>

# Implied Volatility

is the $\sigma$ value that makes the Black-Scholes equation give the same option price as observed on the market. It is often interpreted as the "market uncertainty."

The next cell uses the call option price calculated above as being the "market price." The implied volatility should then be the same as what we used above.

In [11]:
                                #simple (crude) way to solve for implied vol
iv = fzero(σ->OptionBlackSPs(10,10,0.5,0.1,σ)-c1,0.00001,5)

printlnPs("Implied volatility: ",iv,", compare with: $σ")

Implied volatility:      0.400, compare with: 0.4


The next few cells instead some data on options on German government bonds. 

In [12]:
#  LIFFE Bunds option data, trade date April 6, 1994
K = [                        #strike prices; Mx1 vector
      92.00;  94.00;  94.50;  95.00;  95.50;  96.00;  96.50;  97.00;
      97.50;  98.00;  98.50;  99.00;  99.50;  100.0;  100.5;  101.0;
     101.5;  102.0;  102.5;  103.0;  103.5 ];
C = [                        #call prices; Mx1 vector
      5.13;    3.25;    2.83;    2.40;    2.00;    1.64;    1.31;    1.02;
      0.770;   0.570;   0.400;   0.280;   0.190;   0.130;  0.0800;  0.0500;
      0.0400;  0.0300;  0.0200;  0.0100;  0.0100 ];
S = 97.05                #spot price
m = 48/365               #time to expiry in years
y = 0.0                  #Interest rate: LIFFE=>no discounting
N = length(K)

21

In [13]:
iv = fill(NaN,N)
for i = 1:N
  iv[i] = fzero(sigma->OptionBlackSPs(S,K[i],m,y,sigma)-C[i],0.00001,5)
end

println("Strike and iv for data: ")
printmat([K iv])

println("let's plot it")

Strike and iv for data: 
    92.000     0.094
    94.000     0.081
    94.500     0.081
    95.000     0.078
    95.500     0.075
    96.000     0.074
    96.500     0.072
    97.000     0.071
    97.500     0.070
    98.000     0.069
    98.500     0.068
    99.000     0.067
    99.500     0.067
   100.000     0.068
   100.500     0.067
   101.000     0.067
   101.500     0.070
   102.000     0.073
   102.500     0.074
   103.000     0.072
   103.500     0.077

let's plot it


In [14]:
plot1 = plot(K,iv,color=:red,legend=false)
title!("Implied volatility, Bunds options April 6, 1994")
xlabel!("strike price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 92 
 
 
 94 
 
 
 96 
 
 
 98 
 
 
 100 
 
 
 102 
 
 
 0.07 
 
 
 0.08 
 
 
 0.09 
 
 
 Implied volatility, Bunds options April 6, 1994 
 
 
 strike price

# Mean-variance Frontier

Given a vector of average returns ($\mu$) and a variance-covariance matrix ($\Sigma$), the mean-variance frontier show the lowest possible portfolio uncertainty for a given expected portfolio return. It is thus the solution to a quadratic minimization problem. The cells below will use the explicit (matrix) formulas for this solution, but we often have to resort to numerical methods when there are portfolio restrictions.

It is typically plotted with the portfolio standard deviation on the horizontal axis and the portfolio expected return on the vertical axis.

In [15]:
μ = [0.115; 0.095; 0.06]        #expected returns
Σ  = [166  34  58;              #covariance matrix
       34  64   4;
       58   4 100]/100^2                  
Rf = 0.03                       #riskfree return (an interest rate) 

println("μ, Σ and Rf: ")
printmat(μ)
printmat(Σ)
printmat(Rf)

μ, Σ and Rf: 
     0.115
     0.095
     0.060

     0.017     0.003     0.006
     0.003     0.006     0.000
     0.006     0.000     0.010

     0.030



The functions in the next cell use the explicit solution to the minimization problem. They assume there are no portfolio restrictions. 

In [16]:
function MVCalc(mustar,μ,Σ)           #calculates the std of a portfolio on MVF of risky assets
    n    = length(μ)
    oneV = ones(n,1) 
    Σ_1  = inv(Σ)
    A    = (μ'Σ_1*μ)[1]                  #()[1] to make it a scalar, not an 1x1 matrix
    B    = (μ'Σ_1*oneV)[1]
    C    = (oneV'Σ_1*oneV)[1]
    λ    = (C*mustar - B)/(A*C-B^2)
    δ    = (A-B*mustar)/(A*C-B^2)
    w    = Σ_1 *(μ*λ + oneV*δ)
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

function MVCalcRf(mustar,μ,Σ,Rf)           #calculates the std of a portfolio on MVF of (Risky,Riskfree)
    n     = length(μ)
    μe    = μ - Rf                         #expected excess returns            
    Σ_1   = inv(Σ)
    w     = (mustar-Rf)/(μe'Σ_1*μe)[1] * Σ_1 *μe
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

MVCalcRf (generic function with 1 method)

In [17]:
mustar = linspace(Rf,0.15,101)
L      = length(mustar)
StdRp  = fill(NaN,L)
for i = 1:L
    StdRP_i, = MVCalc(mustar[i],μ,Σ)             #, = means skipping the remaining output
    StdRp[i] = StdRP_i
end    

plot3 = plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("MVF, only risky assets")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 MVF, only risky assets 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 512.649,296.378 504.26,293.493 495.923,290.608 487.641,287.723 479.417,284.838 471.254,281.953 463.156,279.068 455.127,276.183 447.17,273.298 439.291,270.413 
 431.493,267.528 423.782,264.643 416.163,261.757 408.642,258.872 401.225,255.987 393.918,253.102 386.728,250.217 379.662,247.332 372.73,244.447 365.938,241.562 
 359.297,238.677 352.815,235.792 346.503,232.907 340.371,230.022 334.431,227.137 328.694,224.252 323.174,221.367 317.881,218.482 312.831,215.597 308.036,212.712 
 303.51,209.827 299.268,206.942 295.324,204.057 291.691,201.172 288.384,198.287 285.414,195.402 282.796,192.517 280.539,189.631 278.655,186.746 277.152,183.861 
 276.037,180.976 275.317,178.091 274.994,175.206 275.07,172.321 275.545,169.436 276.417,166.551 277.681,163.666 279.33,160.781 281.358,157.896 283.754,155.011 
 286.508,152.126 289.608,149.241 293.042,146.356 296.795,143.471 300.855,140.586 305.207,137.701 309.837,134.816 314.731,131.931 319.875,129.046 325.257,126.161 
 330.862,123.276 336.677,120.391 342.692,117.506 348.894,114.62 355.272,111.735 361.816,108.85 368.516,105.965 375.362,103.08 382.347,100.195 389.46,97.3102 
 396.696,94.4252 404.046,91.5402 411.504,88.6551 419.063,85.7701 426.718,82.885 434.463,80 442.292,77.115 450.202,74.2299 458.186,71.3449 466.242,68.4598 
 474.365,65.5748 482.552,62.6898 490.798,59.8047 499.102,56.9197 507.459,54.0346 515.867,51.1496 524.323,48.2646 532.826,45.3795 541.372,42.4945 549.959,39.6094 
 558.585,36.7244 567.249,33.8394 575.948,30.9543 584.681,28.0693 593.446,25.1843 602.242,22.2992 611.067,19.4142 619.921,16.5291 628.801,13.6441 637.706,10.7591 
 646.636,7.87402 
 "/>

In [18]:
StdRpRf  = fill(NaN,L)
for i = 1:L
    StdRpRf_i, = MVCalcRf(mustar[i],μ,Σ,Rf) 
    StdRpRf[i] = StdRpRf_i
end    

plot4 = plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
plot!(StdRpRf*100,mustar*100,legend=nothing,color=:blue)
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("MVF, risky and riskfree assets")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 MVF, risky and riskfree assets 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 512.649,296.378 504.26,293.493 495.923,290.608 487.641,287.723 479.417,284.838 471.254,281.953 463.156,279.068 455.127,276.183 447.17,273.298 439.291,270.413 
 431.493,267.528 423.782,264.643 416.163,261.757 408.642,258.872 401.225,255.987 393.918,253.102 386.728,250.217 379.662,247.332 372.73,244.447 365.938,241.562 
 359.297,238.677 352.815,235.792 346.503,232.907 340.371,230.022 334.431,227.137 328.694,224.252 323.174,221.367 317.881,218.482 312.831,215.597 308.036,212.712 
 303.51,209.827 299.268,206.942 295.324,204.057 291.691,201.172 288.384,198.287 285.414,195.402 282.796,192.517 280.539,189.631 278.655,186.746 277.152,183.861 
 276.037,180.976 275.317,178.091 274.994,175.206 275.07,172.321 275.545,169.436 276.417,166.551 277.681,163.666 279.33,160.781 281.358,157.896 283.754,155.011 
 286.508,152.126 289.608,149.241 293.042,146.356 296.795,143.471 300.855,140.586 305.207,137.701 309.837,134.816 314.731,131.931 319.875,129.046 325.257,126.161 
 330.862,123.276 336.677,120.391 342.692,117.506 348.894,114.62 355.272,111.735 361.816,108.85 368.516,105.965 375.362,103.08 382.347,100.195 389.46,97.3102 
 396.696,94.4252 404.046,91.5402 411.504,88.6551 419.063,85.7701 426.718,82.885 434.463,80 442.292,77.115 450.202,74.2299 458.186,71.3449 466.242,68.4598 
 474.365,65.5748 482.552,62.6898 490.798,59.8047 499.102,56.9197 507.459,54.0346 515.867,51.1496 524.323,48.2646 532.826,45.3795 541.372,42.4945 549.959,39.6094 
 558.585,36.7244 567.249,33.8394 575.948,30.9543 584.681,28.0693 593.446,25.1843 602.242,22.2992 611.067,19.4142 619.921,16.5291 628.801,13.6441 637.706,10.7591 
 646.636,7.87402 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,296.378 44.1997,293.493 49.0293,290.608 53.8589,287.723 58.6886,284.838 63.5182,281.953 68.3478,279.068 73.1774,276.183 78.007,273.298 82.8367,270.413 
 87.6663,267.528 92.4959,264.643 97.3255,261.757 102.155,258.872 106.985,255.987 111.814,253.102 116.644,250.217 121.474,247.332 126.303,244.447 131.133,241.562 
 135.962,238.677 140.792,235.792 145.622,232.907 150.451,230.022 155.281,227.137 160.111,224.252 164.94,221.367 169.77,218.482 174.599,215.597 179.429,212.712 
 184.259,209.827 189.088,206.942 193.918,204.057 198.748,201.172 203.577,198.287 208.407,195.402 213.236,192.517 218.066,189.631 222.896,186.746 227.725,183.861 
 232.555,180.976 237.385,178.091 242.214,175.206 247.044,172.321 251.873,169.436 256.703,166.551 261.533,163.666 266.362,160.781 271.192,157.896 276.021,155.011 
 280.851,152.126 285.681,149.241 290.51,146.356 295.34,143.471 300.17,140.586 304.999,137.701 309.829,134.816 314.658,131.931 319.488,129.046 324.318,126.161 
 329.147,123.276 333.977,120.391 338.807,117.506 343.636,114.62 348.466,111.735 353.295,108.85 358.125,105.965 362.955,103.08 367.784,100.195 372.614,97.3102 
 377.443,94.4252 382.273,91.5402 387.103,88.6551 391.932,85.7701 396.762,82.885 401.592,80 406.421,77.115 411.251,74.2299 416.08,71.3449 420.91,68.4598 
 425.74,65.5748 430.569,62.6898 435.399,59.8047 440.229,56.9197 445.058,54.0346 449.888,51.1496 454.717,48.2646 459.547,45.3795 464.377,42.4945 469.206,39.6094 
 474.036,36.7244 478.866,33.8394 483.695,30.9543 488.525,28.0693 493.354,25.1843 498.184,22.2992 503.014,19.4142 507.843,16.5291 512.673,13.6441 517.502,10.7591 
 522.332,7.87402 
 "/>

# (extra) Mean-variance Frontier without Short Selling

The code below solves (numerically) the following minimization problem 

$\min \text{Var}(R_p) \> \text{ s.t. } \>  \text{E}R_p = \mu^*$,
 
and where we require $w_i\ge 0$
 
The code below calls on
```
quadprog(zeros(N),Sigma,A,'=',b,zeros(N),ones(N),IpoptSolver(print_level=1))
```

This means that we minimize
    $0'w + w' \Sigma w/2$ subject to the restrictions $A w = b$ and $0 \leq w \leq 1$. The equality constraints ($A w = b$) are used to impose $1'w=1$ and $\mu'w = \mu^*$ (se the code below).
    
The packages MathProgBase and Ipopt are documented at https://github.com/JuliaOpt/MathProgBase.jl and https://github.com/JuliaOpt/Ipopt.jl.

In [20]:
using MathProgBase, Ipopt

In [24]:
function MeanVarNoSSPs(mu,Sigma,mustar)
    K = length(mustar)                #MV with no short-sales, numerical minimization
    N = length(mu)
    vv = find((mustar .>= minimum(mu)) .& (mustar .<= maximum(mu)))  #solve only if feasible
    lb = zeros(N,1)              #w >= 0
    ub = ones(N,1)               #w <= 1
    A  = [ones(1,N);mu']         #1'w=1, mu'w = mustar
    w_p   = fill(NaN,(K,N))
    StdRp = fill(NaN,K)                       #to store results in
    for i = vv
        b  = [1;mustar[i]]
        Sol = quadprog(zeros(N),Sigma,A,'=',b,zeros(N),ones(N),IpoptSolver(print_level=1))
        w_i = Sol.sol
        if Sol.status == :Optimal
          w_p[i,:] = w_i'
          StdRp[i] = sqrt((w_i'*Sigma*w_i)[1])
        end
    end
    return StdRp,w_p
end

MeanVarNoSSPs (generic function with 1 method)

In [25]:
Std_no_ss, = MeanVarNoSSPs(μ,Σ,mustar)  

plot(StdRp*100,mustar*100,color=:red,linewidth=2,label="Risky only")
plot!(Std_no_ss*100,mustar*100,color=:green,line=(:dash,2),label="Risky only (no ss) ")
plot!(xlims=(0,15),ylims=(0,15))
title!("MVF (portfolio restrictions)")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 MVF (portfolio restrictions) 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 512.649,296.378 504.26,293.493 495.923,290.608 487.641,287.723 479.417,284.838 471.254,281.953 463.156,279.068 455.127,276.183 447.17,273.298 439.291,270.413 
 431.493,267.528 423.782,264.643 416.163,261.757 408.642,258.872 401.225,255.987 393.918,253.102 386.728,250.217 379.662,247.332 372.73,244.447 365.938,241.562 
 359.297,238.677 352.815,235.792 346.503,232.907 340.371,230.022 334.431,227.137 328.694,224.252 323.174,221.367 317.881,218.482 312.831,215.597 308.036,212.712 
 303.51,209.827 299.268,206.942 295.324,204.057 291.691,201.172 288.384,198.287 285.414,195.402 282.796,192.517 280.539,189.631 278.655,186.746 277.152,183.861 
 276.037,180.976 275.317,178.091 274.994,175.206 275.07,172.321 275.545,169.436 276.417,166.551 277.681,163.666 279.33,160.781 281.358,157.896 283.754,155.011 
 286.508,152.126 289.608,149.241 293.042,146.356 296.795,143.471 300.855,140.586 305.207,137.701 309.837,134.816 314.731,131.931 319.875,129.046 325.257,126.161 
 330.862,123.276 336.677,120.391 342.692,117.506 348.894,114.62 355.272,111.735 361.816,108.85 368.516,105.965 375.362,103.08 382.347,100.195 389.46,97.3102 
 396.696,94.4252 404.046,91.5402 411.504,88.6551 419.063,85.7701 426.718,82.885 434.463,80 442.292,77.115 450.202,74.2299 458.186,71.3449 466.242,68.4598 
 474.365,65.5748 482.552,62.6898 490.798,59.8047 499.102,56.9197 507.459,54.0346 515.867,51.1496 524.323,48.2646 532.826,45.3795 541.372,42.4945 549.959,39.6094 
 558.585,36.7244 567.249,33.8394 575.948,30.9543 584.681,28.0693 593.446,25.1843 602.242,22.2992 611.067,19.4142 619.921,16.5291 628.801,13.6441 637.706,10.7591 
 646.636,7.87402 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#008000; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 407.874,224.252 395.888,221.367 384.208,218.482 372.865,215.597 361.896,212.712 351.339,209.827 341.239,206.942 331.642,204.057 322.6,201.172 314.167,198.287 
 306.401,195.402 299.361,192.517 293.11,189.631 287.705,186.746 283.203,183.861 279.655,180.976 277.104,178.091 275.582,175.206 275.108,172.321 275.545,169.436 
 276.417,166.551 277.681,163.666 279.33,160.781 281.358,157.896 283.754,155.011 286.508,152.126 289.608,149.241 293.042,146.356 296.795,143.471 300.855,140.586 
 305.207,137.701 309.837,134.816 314.731,131.931 319.875,129.046 325.257,126.161 331.501,123.276 340.175,120.391 351.155,117.506 364.21,114.62 379.099,111.735 
 395.592,108.85 413.477,105.965 432.566,103.08 452.689,100.195 473.705,97.3102 495.49,94.4252 
 "/>
 
 
 
 
 Risky only 
 
 
 
 Risky only (no ss)